### Campo elegido para el proyecto: Agricultura

Asistente para analizar artículos de agrícultura

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Import correcto para RecursiveCharacterTextSplitter
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    try:
        from langchain.text_splitter import RecursiveCharacterTextSplitter
    except ImportError:
        from langchain_core.text_splitter import RecursiveCharacterTextSplitter

# Importaciones para cargar documentos
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredHTMLLoader

# Rich para visualización
from rich.console import Console
from rich.panel import Panel
from rich.tree import Tree

# Cargar variables de entorno
load_dotenv()

# Configuración de consola
console = Console()


class LocalEmbeddings:
    """
    Clase para generar embeddings locales usando Sentence Transformers
    Compatible con FAISS y LangChain
    No requiere API keys y funciona offline
    """
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Inicializar modelo de embeddings
        Modelos recomendados:
        - all-MiniLM-L6-v2: Rápido, ligero (80MB), buena calidad
        - paraphrase-multilingual-MiniLM-L12-v2: Mejor para español
        """
        console.print(f"[yellow]Cargando modelo de embeddings: {model_name}...[/yellow]")
        self.model = SentenceTransformer(model_name)
        console.print("[green]✓ Modelo de embeddings cargado[/green]")
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generar embeddings para una lista de documentos"""
        embeddings = self.model.encode(
            texts, 
            show_progress_bar=True, 
            convert_to_numpy=True,
            batch_size=32
        )
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Generar embedding para una consulta"""
        embedding = self.model.encode([text], convert_to_numpy=True)
        return embedding[0].tolist()
    
    def __call__(self, text: str) -> List[float]:
        """Hacer la clase callable para compatibilidad con FAISS"""
        return self.embed_query(text)


class TraceabilityLogger:
    """Clase para mantener trazabilidad completa del sistema"""
    
    def __init__(self):
        self.trace_log = []
        self.start_time = None
        
    def start_trace(self, query: str):
        """Iniciar trazabilidad de una consulta"""
        self.start_time = datetime.now()
        self.trace_log = [{
            "timestamp": self.start_time.isoformat(),
            "event": "query_received",
            "query": query
        }]
        
    def log_event(self, agent: str, action: str, details: Dict[str, Any]):
        """Registrar evento en el log de trazabilidad"""
        self.trace_log.append({
            "timestamp": datetime.now().isoformat(),
            "agent": agent,
            "action": action,
            "details": details
        })
        
    def get_trace(self) -> List[Dict]:
        """Obtener el log completo de trazabilidad"""
        return self.trace_log
    
    def print_trace(self):
        """Imprimir trazabilidad de forma visual"""
        tree = Tree("🔍 [bold blue]Trazabilidad del Sistema[/bold blue]")
        
        for entry in self.trace_log:
            if entry.get("event") == "query_received":
                tree.add(f"[green]Query recibida:[/green] {entry['query']}")
            else:
                agent_node = tree.add(f"[yellow]{entry['agent']}[/yellow]")
                agent_node.add(f"[cyan]Acción:[/cyan] {entry['action']}")
                if entry.get("details"):
                    details_str = json.dumps(entry["details"], indent=2, ensure_ascii=False)
                    agent_node.add(f"[magenta]Detalles:[/magenta]\n{details_str}")
        
        console.print(tree)


class DocumentIndexer:
    """
    Agente 1: Consumo e Indexación de Documentos
    - Carga documentos (PDF/TXT/HTML)
    - Limpia, segmenta en chunks y genera embeddings
    - Indexa en FAISS
    """
    
    def __init__(self, embeddings_model, trace_logger: TraceabilityLogger):
        self.embeddings = embeddings_model
        self.trace_logger = trace_logger
        self.vector_store = None
        self.documents = []
        
    def load_documents(self, directory_path: str) -> List[Document]:
        """Cargar documentos desde un directorio"""
        console.print(f"[bold green]📂 Cargando documentos desde: {directory_path}[/bold green]")
        
        documents = []
        path = Path(directory_path)
        
        # Extensiones soportadas
        loaders_map = {
            '.pdf': PyPDFLoader,
            '.txt': TextLoader,
            '.html': UnstructuredHTMLLoader,
            '.htm': UnstructuredHTMLLoader
        }
        
        file_count = 0
        for file_path in path.rglob('*'):
            if file_path.is_file() and file_path.suffix.lower() in loaders_map:
                try:
                    loader_class = loaders_map[file_path.suffix.lower()]
                    
                    if file_path.suffix.lower() == '.txt':
                        loader = loader_class(str(file_path), encoding='utf-8')
                    else:
                        loader = loader_class(str(file_path))
                    
                    docs = loader.load()
                    documents.extend(docs)
                    file_count += 1
                    
                    if file_count % 10 == 0:
                        console.print(f"  ✓ Cargados {file_count} documentos...")
                        
                except Exception as e:
                    console.print(f"[red]  ✗ Error al cargar {file_path.name}: {str(e)}[/red]")
                    continue
        
        console.print(f"[bold green]✓ Total de documentos cargados: {file_count}[/bold green]")
        
        self.trace_logger.log_event(
            agent="DocumentIndexer",
            action="load_documents",
            details={"total_documents": file_count, "directory": directory_path}
        )
        
        self.documents = documents
        return documents
    
    def process_and_index(self, documents: List[Document], chunk_size: int = 1000, chunk_overlap: int = 200):
        """Procesar documentos: chunking + embeddings + indexación en FAISS"""
        console.print("[bold yellow]🔨 Procesando documentos...[/bold yellow]")
        
        # Dividir en chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )
        
        chunks = text_splitter.split_documents(documents)
        console.print(f"  ✓ Documentos divididos en {len(chunks)} chunks")
        
        # Crear vector store con FAISS
        console.print("[bold yellow]🧮 Generando embeddings e indexando en FAISS...[/bold yellow]")
        self.vector_store = FAISS.from_documents(chunks, self.embeddings)
        console.print("[bold green]✓ Indexación completada[/bold green]")
        
        self.trace_logger.log_event(
            agent="DocumentIndexer",
            action="process_and_index",
            details={
                "total_chunks": len(chunks),
                "chunk_size": chunk_size,
                "chunk_overlap": chunk_overlap
            }
        )
        
        return self.vector_store
    
    def save_index(self, path: str = "faiss_index"):
        """Guardar índice FAISS en disco"""
        if self.vector_store:
            self.vector_store.save_local(path)
            console.print(f"[green]✓ Índice guardado en: {path}[/green]")
    
    def load_index(self, path: str = "faiss_index"):
        """Cargar índice FAISS desde disco"""
        try:
            self.vector_store = FAISS.load_local(
                path, 
                self.embeddings,
                allow_dangerous_deserialization=True
            )
            console.print(f"[green]✓ Índice cargado desde: {path}[/green]")
            return self.vector_store
        except Exception as e:
            console.print(f"[red]✗ Error al cargar índice: {str(e)}[/red]")
            return None


class QueryClassifier:
    """
    Agente 2: Clasificador de Consultas
    Clasifica la intención del usuario en 4 categorías:
    - búsqueda: Consulta específica en documentos
    - resumen: Resumen de documentos
    - comparación: Comparar documentos
    - general: Pregunta general sin necesidad de RAG
    
    Usa Groq Llama 3.3 70B para interpretación profunda del lenguaje
    """
    
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        # Groq Llama 3.3 70B para interpretación profunda del lenguaje
        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.1,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def classify(self, query: str) -> Dict[str, Any]:
        """Clasificar la intención de la consulta del usuario"""
        
        classification_prompt = ChatPromptTemplate.from_messages([
            ("system", """Eres un clasificador de consultas experto. Debes clasificar la intención del usuario en una de estas categorías:

1. "busqueda": El usuario solicita información específica, hechos o datos contenidos en documentos
2. "resumen": El usuario quiere un resumen de uno o varios documentos
3. "comparacion": El usuario quiere comparar o contrastar información de diferentes documentos
4. "general": El usuario hace una pregunta general que no requiere buscar en documentos específicos

Ejemplos:
- "¿Cuál es la capital de Francia?" -> general
- "¿Qué dice el documento sobre las causas de la guerra?" -> busqueda
- "Resume el contenido del artículo sobre IA" -> resumen
- "Compara las conclusiones de estos dos estudios" -> comparacion
- "Explícame qué es la fotosíntesis" -> general
- "¿Qué información hay sobre cambio climático?" -> busqueda

Responde SOLO con un JSON en este formato exacto:
{{"intencion": "busqueda|resumen|comparacion|general", "confianza": 0.95, "razonamiento": "breve explicación"}}"""),
            ("user", "{query}")
        ])
        
        chain = classification_prompt | self.llm | StrOutputParser()
        response = chain.invoke({"query": query})
        
        # Parsear respuesta
        try:
            # Limpiar respuesta si tiene markdown
            response_clean = response.strip()
            if response_clean.startswith("```json"):
                response_clean = response_clean[7:]
            if response_clean.endswith("```"):
                response_clean = response_clean[:-3]
            
            result = json.loads(response_clean.strip())
            
            self.trace_logger.log_event(
                agent="QueryClassifier",
                action="classify_intent",
                details={
                    "query": query,
                    "clasificacion": result
                }
            )
            
            console.print(Panel(
                f"[bold cyan]Intención:[/bold cyan] {result['intencion']}\n"
                f"[bold cyan]Confianza:[/bold cyan] {result['confianza']}\n"
                f"[bold cyan]Razonamiento:[/bold cyan] {result['razonamiento']}",
                title="🎯 Clasificación de Query"
            ))
            
            return result
            
        except json.JSONDecodeError:
            console.print("[red]Error al parsear clasificación, usando 'busqueda' por defecto[/red]")
            return {
                "intencion": "busqueda",
                "confianza": 0.5,
                "razonamiento": "Error en clasificación, asumiendo búsqueda"
            }


class SemanticRetriever:
    """
    Agente 3: Recuperador Semántico
    - Ejecuta búsqueda de similaridad semántica en FAISS
    - Selecciona documentos más relevantes
    
    Usa Groq Llama 3.1 8B para optimizar velocidad de recuperación
    """
    
    def __init__(self, vector_store: FAISS, trace_logger: TraceabilityLogger):
        self.vector_store = vector_store
        self.trace_logger = trace_logger
        # Groq Llama 3.1 8B para velocidad de recuperación
        self.llm = ChatGroq(
            model="llama-3.1-8b-instant",
            temperature=0,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def retrieve(self, query: str, k: int = 5) -> List[Document]:
        """Recuperar documentos relevantes mediante búsqueda semántica"""
        
        console.print(f"[bold yellow]🔎 Buscando documentos relevantes (top-{k})...[/bold yellow]")
        
        # Búsqueda de similaridad en FAISS
        relevant_docs = self.vector_store.similarity_search(query, k=k)
        
        self.trace_logger.log_event(
            agent="SemanticRetriever",
            action="retrieve_documents",
            details={
                "query": query,
                "num_docs_retrieved": len(relevant_docs),
                "doc_sources": [doc.metadata.get('source', 'unknown') for doc in relevant_docs]
            }
        )
        
        console.print(f"[green]✓ Recuperados {len(relevant_docs)} documentos relevantes[/green]")
        
        return relevant_docs


class RAGResponseGenerator:
    """
    Agente 4: Generador de Respuestas con RAG
    - Construye respuesta combinando query + contexto recuperado
    - Produce respuestas justificadas con citas
    
    Usa Groq Llama 3.3 70B para generar respuestas rápidas basadas en contexto
    """
    
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        # Groq Llama 3.3 70B para respuestas de alta calidad
        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.3,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def generate_response(self, query: str, context_docs: List[Document], intent: str) -> str:
        """Generar respuesta usando RAG"""
        
        console.print("[bold yellow]✍️ Generando respuesta con RAG...[/bold yellow]")
        
        # Preparar contexto
        context = "\n\n".join([
            f"--- Documento {i+1} (Fuente: {doc.metadata.get('source', 'desconocida')}) ---\n{doc.page_content}"
            for i, doc in enumerate(context_docs)
        ])
        
        # Prompt según intención
        if intent == "resumen":
            system_prompt = """Eres un asistente experto en resumir documentos. 
            Genera un resumen claro y conciso del contenido proporcionado.
            Incluye las ideas principales y cita las fuentes cuando sea apropiado."""
        elif intent == "comparacion":
            system_prompt = """Eres un asistente experto en análisis comparativo.
            Compara y contrasta la información de los documentos proporcionados.
            Identifica similitudes, diferencias y cita las fuentes de cada afirmación."""
        else:  # búsqueda
            system_prompt = """Eres un asistente experto en responder preguntas basándote en documentos.
            Responde la pregunta del usuario usando ÚNICAMENTE la información del contexto proporcionado.
            IMPORTANTE: Cita las fuentes de donde obtienes la información.
            Si la información no está en el contexto, indícalo claramente."""
        
        rag_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("user", """Contexto de los documentos:
{context}

Pregunta del usuario: {query}

Responde de forma clara, precisa y justificada con citas a las fuentes.""")
        ])
        
        chain = rag_prompt | self.llm | StrOutputParser()
        response = chain.invoke({"context": context, "query": query})
        
        self.trace_logger.log_event(
            agent="RAGResponseGenerator",
            action="generate_response",
            details={
                "query": query,
                "intent": intent,
                "num_context_docs": len(context_docs),
                "response_length": len(response)
            }
        )
        
        console.print("[green]✓ Respuesta generada[/green]")
        
        return response


class ResponseVerifier:
    """
    Agente 5: Verificador/Crítico de Respuestas
    - Evalúa si la respuesta está respaldada por el contexto
    - Verifica coherencia y evita alucinaciones
    - Valida cumplimiento de requerimientos
    
    Usa Groq Llama 3.3 70B para razonamiento y validación compleja
    """
    
    def __init__(self, trace_logger: TraceabilityLogger):
        self.trace_logger = trace_logger
        # Groq Llama 3.3 70B para razonamiento y validación compleja
        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.1,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def verify(self, query: str, response: str, context_docs: List[Document]) -> Dict[str, Any]:
        """Verificar calidad y coherencia de la respuesta"""
        
        console.print("[bold yellow]✅ Verificando respuesta...[/bold yellow]")
        
        context = "\n\n".join([doc.page_content for doc in context_docs])
        
        verification_prompt = ChatPromptTemplate.from_messages([
            ("system", """Eres un evaluador crítico de respuestas. Tu trabajo es verificar:

1. ¿La respuesta está respaldada por el contexto proporcionado?
2. ¿La respuesta es coherente y lógica?
3. ¿Hay signos de alucinación (información inventada no presente en el contexto)?
4. ¿La respuesta responde adecuadamente a la pregunta del usuario?

Responde con un JSON en este formato:
{{
    "es_valida": true/false,
    "confianza": 0.0-1.0,
    "problemas_detectados": ["lista de problemas si existen"],
    "sugerencias_mejora": ["sugerencias si es_valida es false"],
    "razonamiento": "explicación de tu evaluación"
}}"""),
            ("user", """Contexto:
{context}

Pregunta del usuario: {query}

Respuesta a evaluar: {response}

Evalúa la respuesta.""")
        ])
        
        chain = verification_prompt | self.llm | StrOutputParser()
        verification_result = chain.invoke({
            "context": context,
            "query": query,
            "response": response
        })
        
        try:
            # Limpiar respuesta
            verification_clean = verification_result.strip()
            if verification_clean.startswith("```json"):
                verification_clean = verification_clean[7:]
            if verification_clean.endswith("```"):
                verification_clean = verification_clean[:-3]
            
            result = json.loads(verification_clean.strip())
            
            self.trace_logger.log_event(
                agent="ResponseVerifier",
                action="verify_response",
                details=result
            )
            
            if result["es_valida"]:
                console.print("[bold green]✓ Respuesta verificada exitosamente[/bold green]")
            else:
                console.print("[bold red]✗ Respuesta requiere mejoras[/bold red]")
                console.print(f"[yellow]Problemas: {', '.join(result['problemas_detectados'])}[/yellow]")
            
            return result
            
        except json.JSONDecodeError:
            console.print("[red]Error al parsear verificación[/red]")
            return {
                "es_valida": True,
                "confianza": 0.5,
                "problemas_detectados": [],
                "sugerencias_mejora": [],
                "razonamiento": "Error en verificación, aceptando respuesta"
            }


class Orchestrator:
    """
    Agente Orquestador Principal
    - Administra el flujo completo del sistema
    - Determina qué agente ejecutar según la consulta
    - Coordina la interacción entre agentes
    """
    
    def __init__(
        self,
        indexer: DocumentIndexer,
        classifier: QueryClassifier,
        retriever: SemanticRetriever,
        generator: RAGResponseGenerator,
        verifier: ResponseVerifier,
        trace_logger: TraceabilityLogger
    ):
        self.indexer = indexer
        self.classifier = classifier
        self.retriever = retriever
        self.generator = generator
        self.verifier = verifier
        self.trace_logger = trace_logger
        
        # LLM para decisiones del orquestador (Llama 3.1 8B por velocidad)
        self.llm = ChatGroq(
            model="llama-3.1-8b-instant",
            temperature=0,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
    def process_query(self, query: str, max_retries: int = 2) -> Dict[str, Any]:
        """Procesar consulta del usuario orquestando todos los agentes"""
        
        console.print(Panel(
            f"[bold white]{query}[/bold white]",
            title="📝 Query del Usuario",
            border_style="blue"
        ))
        
        # Iniciar trazabilidad
        self.trace_logger.start_trace(query)
        
        # Paso 1: Clasificar intención
        classification = self.classifier.classify(query)
        intent = classification["intencion"]
        
        self.trace_logger.log_event(
            agent="Orchestrator",
            action="route_query",
            details={"intent": intent}
        )
        
        # Paso 2: Rutear según intención
        if intent == "general":
            # Respuesta directa sin RAG
            return self._handle_general_query(query)
        else:
            # Requiere RAG: búsqueda, resumen o comparación
            return self._handle_rag_query(query, intent, max_retries)
    
    def _handle_general_query(self, query: str) -> Dict[str, Any]:
        """Manejar consulta general sin necesidad de RAG"""
        
        console.print("[bold cyan]💬 Procesando consulta general (sin RAG)...[/bold cyan]")
        
        # Usar Groq Llama 3.3 70B para respuestas generales de calidad
        llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.7,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        
        general_prompt = ChatPromptTemplate.from_messages([
            ("system", "Eres un asistente útil y conocedor. Responde las preguntas del usuario de manera clara y precisa."),
            ("user", "{query}")
        ])
        
        chain = general_prompt | llm | StrOutputParser()
        response = chain.invoke({"query": query})
        
        self.trace_logger.log_event(
            agent="Orchestrator",
            action="handle_general_query",
            details={"response_length": len(response)}
        )
        
        return {
            "query": query,
            "intent": "general",
            "response": response,
            "documents_used": [],
            "verification": {"es_valida": True, "confianza": 1.0},
            "trace": self.trace_logger.get_trace()
        }
    
    def _handle_rag_query(self, query: str, intent: str, max_retries: int) -> Dict[str, Any]:
        """Manejar consulta que requiere RAG"""
        
        console.print(f"[bold cyan]🔄 Procesando consulta con RAG (intención: {intent})...[/bold cyan]")
        
        # Paso 1: Recuperar documentos relevantes
        k = 5 if intent == "busqueda" else 10  # Más docs para resumen/comparación
        relevant_docs = self.retriever.retrieve(query, k=k)
        
        if not relevant_docs:
            return {
                "query": query,
                "intent": intent,
                "response": "No se encontraron documentos relevantes para responder a tu consulta.",
                "documents_used": [],
                "verification": {"es_valida": False, "confianza": 0.0},
                "trace": self.trace_logger.get_trace()
            }
        
        # Paso 2: Generar respuesta con RAG (con loop de verificación)
        for attempt in range(max_retries + 1):
            console.print(f"[cyan]Intento {attempt + 1}/{max_retries + 1} de generación...[/cyan]")
            
            response = self.generator.generate_response(query, relevant_docs, intent)
            
            # Paso 3: Verificar respuesta
            verification = self.verifier.verify(query, response, relevant_docs)
            
            if verification["es_valida"] or attempt == max_retries:
                # Respuesta válida o se alcanzó máximo de intentos
                return {
                    "query": query,
                    "intent": intent,
                    "response": response,
                    "documents_used": [doc.metadata.get('source', 'unknown') for doc in relevant_docs],
                    "verification": verification,
                    "attempts": attempt + 1,
                    "trace": self.trace_logger.get_trace()
                }
            
            console.print("[yellow]⚠️ Respuesta no válida, regenerando...[/yellow]")
        
        # Esto no debería alcanzarse nunca
        return {
            "query": query,
            "intent": intent,
            "response": response,
            "documents_used": [doc.metadata.get('source', 'unknown') for doc in relevant_docs],
            "verification": verification,
            "trace": self.trace_logger.get_trace()
        }


def main():
    """Función principal del sistema"""
    
    console.print(Panel(
        "[bold cyan]Sistema Agentic AI Multi-Agente[/bold cyan]\n"
        "[white]Universidad Nacional de Colombia - PLN[/white]\n"
        "[yellow]Procesamiento y Análisis Inteligente de Documentos[/yellow]",
        title="🤖 Bienvenido",
        border_style="green"
    ))
    
    # Verificar variable de entorno
    if not os.getenv("GROQ_API_KEY"):
        console.print("[bold red]Error: GROQ_API_KEY no configurada[/bold red]")
        console.print("[yellow]Por favor, añade tu API key de Groq al archivo .env[/yellow]")
        console.print("[yellow]Obténla en: https://console.groq.com/keys[/yellow]")
        return
    
    # Inicializar componentes
    console.print("\n[bold cyan]Inicializando sistema...[/bold cyan]")
    
    # Embeddings locales (no requiere API)
    embeddings = LocalEmbeddings(model_name="all-MiniLM-L6-v2")
    
    trace_logger = TraceabilityLogger()
    
    # Inicializar agentes
    indexer = DocumentIndexer(embeddings, trace_logger)
    
    # Obtener rutas desde .env o usar defaults relativos
    documents_path = os.getenv("DOCUMENTS_PATH", "./data")
    index_path = os.getenv("FAISS_INDEX_PATH", "./faiss_index")
    
    # Convertir a ruta absoluta para mejor manejo
    documents_path = os.path.abspath(documents_path)
    index_path = os.path.abspath(index_path)
    
    console.print(f"[cyan]Ruta de documentos: {documents_path}[/cyan]")
    console.print(f"[cyan]Ruta de índice: {index_path}[/cyan]")
    
    # Cargar o crear índice
    vector_store = indexer.load_index(index_path)
    
    if vector_store is None:
        # Verificar que existe el directorio de documentos
        if not os.path.exists(documents_path):
            console.print(f"[bold red]Error: No se encuentra el directorio de documentos:[/bold red]")
            console.print(f"[yellow]{documents_path}[/yellow]")
            console.print(f"[cyan]Crea la carpeta 'data' en el directorio del proyecto y coloca tus documentos allí[/cyan]")
            return
        
        # Cargar y procesar documentos
        documents = indexer.load_documents(documents_path)
        
        if not documents:
            console.print("[bold red]No se encontraron documentos para procesar[/bold red]")
            console.print("[yellow]Formatos soportados: PDF, TXT, HTML[/yellow]")
            return
        
        vector_store = indexer.process_and_index(documents)
        indexer.save_index(index_path)
    
    # Inicializar resto de agentes
    classifier = QueryClassifier(trace_logger)
    retriever = SemanticRetriever(vector_store, trace_logger)
    generator = RAGResponseGenerator(trace_logger)
    verifier = ResponseVerifier(trace_logger)
    
    # Inicializar orquestador
    orchestrator = Orchestrator(
        indexer, classifier, retriever, generator, verifier, trace_logger
    )
    
    console.print("[bold green]✓ Sistema inicializado exitosamente[/bold green]\n")
    
    # Loop interactivo
    while True:
        console.print("\n" + "="*80)
        query = console.input("[bold cyan]👤 Ingresa tu consulta (o 'salir' para terminar): [/bold cyan]")
        
        if query.lower() in ['salir', 'exit', 'quit']:
            console.print("[bold green]¡Hasta luego![/bold green]")
            break
        
        if not query.strip():
            continue
        
        # Procesar consulta
        result = orchestrator.process_query(query)
        
        # Mostrar respuesta
        console.print("\n")
        console.print(Panel(
            f"[bold white]{result['response']}[/bold white]",
            title="🤖 Respuesta del Sistema",
            border_style="green"
        ))
        
        # Mostrar información adicional
        if result.get("documents_used"):
            console.print(f"\n[cyan]📚 Documentos utilizados: {len(result['documents_used'])}[/cyan]")
        
        console.print(f"[cyan]🎯 Intención detectada: {result['intent']}[/cyan]")
        
        if result.get("verification"):
            ver = result["verification"]
            console.print(f"[cyan]✅ Confianza de verificación: {ver.get('confianza', 0):.2f}[/cyan]")
        
        # Mostrar trazabilidad
        if console.input("\n[yellow]¿Mostrar trazabilidad completa? (s/n): [/yellow]").lower() == 's':
            trace_logger.print_trace()


if __name__ == "__main__":
    main()

╭───────────────────────────────────────────────── 🤖 Bienvenido ─────────────────────────────────────────────────╮
│ Sistema Agentic AI Multi-Agente                                                                                 │
│ Universidad Nacional de Colombia - PLN                                                                          │
│ Procesamiento y Análisis Inteligente de Documentos                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Inicializando sistema...

Cargando modelo de embeddings: all-MiniLM-L6-v2...

✓ Modelo de embeddings cargado

Ruta de documentos: c:\GitHub\avioa\Sistema-multiagente-para-el-procesamiento-y-an-lisis-de-documentos-\data

...

✅ Confianza de verificación: 1.00

¿Mostrar trazabilidad completa? (s/n): 

================================================================================

👤 Ingresa tu consulta (o 'salir' para terminar): 

¡Hasta luego!

In [26]:
import os
from langchain_groq import ChatGroq

def build_llm_for_component(component: str):
    """
    Devuelve un LLM configurado según:
      - LLM_TIER: 'cheap' o 'full'
      - componente: 'classifier' o 'generator'

    Se usan modelos de Groq:
      - cheap -> llama-3.1-8b-instant (más barato)
      - full  -> llama-3.3-70b-versatile (más potente)

    Puedes cambiar estos modelos si quieres usar Gemini, pero la interfaz se mantiene.
    """
    tier = os.getenv("LLM_TIER", "cheap").lower()  # por defecto cheap para pruebas
    api_key = os.getenv("GROQ_API_KEY")

    if not api_key:
        raise RuntimeError("GROQ_API_KEY no está configurada en el entorno")

    if tier == "full":
        model_name = "llama-3.3-70b-versatile"
        if component == "classifier":
            max_tokens = 64
        else:  # generator
            max_tokens = 512
    else:
        # Modo económico por defecto
        model_name = "llama-3.1-8b-instant"
        if component == "classifier":
            max_tokens = 32
        else:  # generator
            max_tokens = 256

    return ChatGroq(
        model=model_name,
        api_key=api_key,
        temperature=0.0,
        max_tokens=max_tokens,
    )


In [ ]:
"""
================================================================================
SUITE DE PRUEBAS ROBUSTAS - SISTEMA MULTI-AGENTE RAG
================================================================================
Arquitectura HÍBRIDA conforme a requisitos del trabajo:

┌─────────────────────────────────────────────────────────────────────────────┐
│  COMPONENTE          │  PRIMARIO              │  FALLBACK                   │
├─────────────────────────────────────────────────────────────────────────────┤
│  Clasificador        │  Groq Llama 3.3 70B    │  Gemini 2.0 Flash          │
│  Recuperador         │  SentenceTransformer   │  (local, sin API)          │
│  Generador RAG       │  Gemini 2.0 Flash      │  Groq Llama 3.3 70B        │
│  Verificador         │  Groq Llama 3.3 70B    │  Gemini 2.0 Flash          │
│  General QA          │  Gemini 2.0 Flash      │  Groq Llama 3.3 70B        │
└─────────────────────────────────────────────────────────────────────────────┘

ESTRATEGIA:
- Groq para tareas de pocos tokens (clasificación, verificación)
- Gemini para generación de respuestas largas
- Fallback automático si el primario falla por rate limit
- Todos los agentes usan LLMs reales (cumple requisitos del trabajo)

Autor: Sistema Multi-Agente PLN - Universidad Nacional de Colombia
================================================================================
"""

import json
import os
import time
import requests
from datetime import datetime
from typing import List, Dict, Any, Optional, Tuple
from rich.console import Console
from rich.table import Table
from rich.panel import Panel

console = Console()

# =============================================================================
# CONFIGURACIÓN GLOBAL
# =============================================================================
CONFIG = {
    # Modelos
    "GROQ_MODEL_LARGE": "llama-3.3-70b-versatile",
    "GROQ_MODEL_FAST": "llama-3.1-8b-instant",
    "GEMINI_MODEL": "gemini-2.0-flash",
    
    # Embeddings
    "EMBEDDING_MODEL": "all-MiniLM-L6-v2",
    
    # Parámetros RAG
    "RETRIEVAL_K_SEARCH": 5,
    "RETRIEVAL_K_SUMMARY": 8,
    "RETRIEVAL_K_COMPARE": 6,
    "MAX_CONTEXT_CHARS": 3000,
    
    # Rate limiting
    "DELAY_BETWEEN_TESTS": 3,      # Delay entre pruebas
    "GROQ_RETRY_DELAY": 2,         # Retry para Groq
    "GEMINI_RETRY_DELAY": 10,      # Retry para Gemini (más conservador)
    "MAX_RETRIES": 3,
}


# =============================================================================
# CLIENTE GEMINI (REST API)
# =============================================================================
class GeminiClient:
    """Cliente directo para Gemini API usando REST con retry."""
    
    def __init__(self, api_key: str = None, model: str = None):
        self.api_key = api_key or os.getenv("GOOGLE_API_KEY")
        self.model = model or CONFIG["GEMINI_MODEL"]
        self.base_url = "https://generativelanguage.googleapis.com/v1beta"
        self.last_call_time = 0
        self.min_interval = 4
        
        if not self.api_key:
            raise ValueError("GOOGLE_API_KEY no configurada")
    
    def _wait_if_needed(self):
        elapsed = time.time() - self.last_call_time
        if elapsed < self.min_interval:
            wait = self.min_interval - elapsed
            time.sleep(wait)
    
    def generate(self, prompt: str, system_prompt: str = None, temperature: float = 0.3, max_tokens: int = 1024) -> Tuple[str, bool]:
        """
        Generar texto con Gemini.
        Retorna: (respuesta, éxito)
        """
        self._wait_if_needed()
        
        url = f"{self.base_url}/models/{self.model}:generateContent?key={self.api_key}"
        
        full_prompt = f"{system_prompt}\n\n{prompt}" if system_prompt else prompt
        
        payload = {
            "contents": [{"parts": [{"text": full_prompt}]}],
            "generationConfig": {
                "temperature": temperature,
                "maxOutputTokens": max_tokens,
            }
        }
        
        for attempt in range(CONFIG["MAX_RETRIES"]):
            try:
                response = requests.post(url, headers={"Content-Type": "application/json"}, json=payload, timeout=60)
                self.last_call_time = time.time()
                
                if response.status_code == 200:
                    data = response.json()
                    if "candidates" in data and data["candidates"]:
                        text = data["candidates"][0]["content"]["parts"][0]["text"]
                        return text, True
                    return "Error: Respuesta vacía", False
                
                elif response.status_code == 429:
                    wait_time = CONFIG["GEMINI_RETRY_DELAY"] * (2 ** attempt)
                    console.print(f"[yellow]⏳ Gemini rate limit, esperando {wait_time}s...[/yellow]")
                    time.sleep(wait_time)
                    continue
                else:
                    error_msg = response.text[:100]
                    if attempt < CONFIG["MAX_RETRIES"] - 1:
                        time.sleep(CONFIG["GEMINI_RETRY_DELAY"])
                        continue
                    return f"Error Gemini: {error_msg}", False
                    
            except Exception as e:
                if attempt < CONFIG["MAX_RETRIES"] - 1:
                    time.sleep(5)
                    continue
                return f"Error: {str(e)[:100]}", False
        
        return "Error: Máximo de reintentos Gemini", False


# =============================================================================
# CLIENTE GROQ CON FALLBACK
# =============================================================================
class GroqWithFallback:
    """
    Cliente Groq con fallback automático a Gemini.
    Usa LangChain ChatGroq para mantener compatibilidad.
    """
    
    def __init__(self, gemini_client: GeminiClient, model: str = None, temperature: float = 0.1):
        from langchain_groq import ChatGroq
        from langchain_core.prompts import ChatPromptTemplate
        from langchain_core.output_parsers import StrOutputParser
        
        self.groq_api_key = os.getenv("GROQ_API_KEY")
        self.model = model or CONFIG["GROQ_MODEL_LARGE"]
        self.temperature = temperature
        self.gemini = gemini_client
        
        if self.groq_api_key:
            self.llm = ChatGroq(
                model=self.model,
                temperature=temperature,
                groq_api_key=self.groq_api_key
            )
        else:
            self.llm = None
            console.print("[yellow]⚠ GROQ_API_KEY no configurada, usando solo Gemini[/yellow]")
    
    def invoke(self, system_prompt: str, user_prompt: str, max_tokens: int = 256) -> Tuple[str, str]:
        """
        Invocar LLM con fallback.
        Retorna: (respuesta, modelo_usado)
        """
        # Intentar con Groq primero
        if self.llm:
            try:
                from langchain_core.prompts import ChatPromptTemplate
                from langchain_core.output_parsers import StrOutputParser
                
                prompt = ChatPromptTemplate.from_messages([
                    ("system", system_prompt),
                    ("user", user_prompt)
                ])
                chain = prompt | self.llm | StrOutputParser()
                response = chain.invoke({})
                return response, f"Groq ({self.model})"
                
            except Exception as e:
                error_str = str(e).lower()
                if "rate" in error_str or "limit" in error_str or "429" in error_str or "quota" in error_str:
                    console.print(f"[yellow]⚠ Groq rate limit, usando Gemini fallback...[/yellow]")
                else:
                    console.print(f"[yellow]⚠ Error Groq: {str(e)[:50]}, usando Gemini fallback...[/yellow]")
        
        # Fallback a Gemini
        response, success = self.gemini.generate(user_prompt, system_prompt, self.temperature, max_tokens)
        if success:
            return response, "Gemini (fallback)"
        else:
            return response, "ERROR"


# =============================================================================
# EMBEDDINGS LOCALES
# =============================================================================
class LocalEmbeddingsEngine:
    """Motor de embeddings 100% local usando SentenceTransformer."""
    
    def __init__(self, model_name: str = None):
        from sentence_transformers import SentenceTransformer
        model_name = model_name or CONFIG["EMBEDDING_MODEL"]
        console.print(f"[yellow]⚙️ Cargando embeddings: {model_name}[/yellow]")
        self.model = SentenceTransformer(model_name)
        self._dim = self.model.get_sentence_embedding_dimension()
        console.print(f"[green]✓ Embeddings cargados (dim={self._dim})[/green]")
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_numpy=True, show_progress_bar=False).tolist()
    
    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text], convert_to_numpy=True)[0].tolist()
    
    def __call__(self, text: str) -> List[float]:
        return self.embed_query(text)


# =============================================================================
# CLASIFICADOR CON LLM (Groq + Gemini fallback)
# =============================================================================
class HybridIntentClassifier:
    """
    Clasificador de intenciones usando LLM real.
    Primario: Groq Llama 3.3 70B (pocos tokens)
    Fallback: Gemini 2.0 Flash
    """
    
    SYSTEM_PROMPT = """Eres un clasificador de consultas experto. Clasifica la intención del usuario en:

1. "busqueda": Solicita información específica de documentos
2. "resumen": Quiere un resumen de documentos
3. "comparacion": Quiere comparar información de diferentes documentos
4. "general": Pregunta general que no requiere buscar en documentos

Ejemplos:
- "¿Qué es la fotosíntesis?" -> general
- "¿Qué dice el documento sobre plagas?" -> busqueda
- "Resume el artículo sobre riego" -> resumen
- "Compara métodos de fertilización" -> comparacion

Responde SOLO con JSON: {"intencion": "...", "confianza": 0.95, "razonamiento": "..."}"""
    
    def __init__(self, trace_logger, groq_client: GroqWithFallback):
        self.trace_logger = trace_logger
        self.groq = groq_client
    
    def classify(self, query: str) -> Dict[str, Any]:
        """Clasificar intención usando LLM."""
        console.print(f"[cyan]🎯 Clasificando intención (LLM)...[/cyan]")
        
        response, model_used = self.groq.invoke(
            self.SYSTEM_PROMPT,
            query,
            max_tokens=100
        )
        
        try:
            # Limpiar respuesta
            response_clean = response.strip()
            if response_clean.startswith("```json"):
                response_clean = response_clean[7:]
            if response_clean.startswith("```"):
                response_clean = response_clean[3:]
            if response_clean.endswith("```"):
                response_clean = response_clean[:-3]
            
            result = json.loads(response_clean.strip())
            result["modelo_usado"] = model_used
            
            console.print(f"[green]✓ Clasificado:[/green] [bold]{result['intencion']}[/bold] (conf: {result.get('confianza', 0):.2f}) [{model_used}]")
            
            self.trace_logger.log_event(
                agent="HybridIntentClassifier",
                action="classify",
                details={"query": query[:50], "result": result}
            )
            
            return result
            
        except json.JSONDecodeError:
            console.print(f"[yellow]⚠ Error parsing, usando heurística de respaldo[/yellow]")
            # Heurística de respaldo si el JSON falla
            query_lower = query.lower()
            if any(kw in query_lower for kw in ["resume", "resumen", "sintetiza"]):
                intent = "resumen"
            elif any(kw in query_lower for kw in ["compara", "contrasta", "diferencia"]):
                intent = "comparacion"
            elif any(kw in query_lower for kw in ["qué es", "explica", "define"]):
                intent = "general"
            else:
                intent = "busqueda"
            
            result = {"intencion": intent, "confianza": 0.6, "razonamiento": "Heurística de respaldo", "modelo_usado": "heurística"}
            self.trace_logger.log_event(agent="HybridIntentClassifier", action="classify_fallback", details=result)
            return result


# =============================================================================
# RECUPERADOR SEMÁNTICO
# =============================================================================
class SemanticDocumentRetriever:
    """Recuperador de documentos usando FAISS + embeddings locales."""
    
    def __init__(self, vector_store, trace_logger):
        self.vector_store = vector_store
        self.trace_logger = trace_logger
    
    def retrieve(self, query: str, intent: str) -> List:
        """Recuperar documentos relevantes según intención."""
        k_map = {
            "busqueda": CONFIG["RETRIEVAL_K_SEARCH"],
            "resumen": CONFIG["RETRIEVAL_K_SUMMARY"],
            "comparacion": CONFIG["RETRIEVAL_K_COMPARE"],
        }
        k = k_map.get(intent, 5)
        
        console.print(f"[yellow]🔎 Recuperando top-{k} documentos...[/yellow]")
        
        try:
            docs = self.vector_store.similarity_search(query, k=k)
            console.print(f"[green]✓ {len(docs)} documentos recuperados[/green]")
            self.trace_logger.log_event(
                agent="SemanticDocumentRetriever",
                action="retrieve",
                details={"query": query[:50], "k": k, "retrieved": len(docs)}
            )
            return docs
        except Exception as e:
            console.print(f"[red]✗ Error: {e}[/red]")
            return []


# =============================================================================
# GENERADOR RAG (Gemini primario, Groq fallback)
# =============================================================================
class HybridRAGGenerator:
    """
    Generador de respuestas RAG.
    Primario: Gemini 2.0 Flash (para respuestas largas)
    Fallback: Groq Llama 3.3 70B
    """
    
    SYSTEM_PROMPTS = {
        "busqueda": """Eres un asistente experto en análisis de documentos agrícolas.
Responde basándote ÚNICAMENTE en el contexto proporcionado.
Sé preciso y cita las fuentes. Si la información no está en el contexto, indícalo.""",
        
        "resumen": """Eres un experto en síntesis de información técnica agrícola.
Genera un resumen estructurado con las ideas principales.
Usa viñetas cuando sea apropiado.""",
        
        "comparacion": """Eres un analista experto en agricultura comparativa.
Compara y contrasta la información identificando similitudes y diferencias.
Cita las fuentes de cada afirmación."""
    }
    
    def __init__(self, trace_logger, gemini_client: GeminiClient, groq_client: GroqWithFallback):
        self.trace_logger = trace_logger
        self.gemini = gemini_client
        self.groq = groq_client
    
    def generate(self, query: str, docs: List, intent: str) -> Tuple[str, str]:
        """
        Generar respuesta RAG.
        Retorna: (respuesta, modelo_usado)
        """
        console.print(f"[yellow]✍️ Generando respuesta RAG...[/yellow]")
        
        # Construir contexto
        context_parts = []
        total_chars = 0
        for i, doc in enumerate(docs):
            content = doc.page_content[:800]
            if total_chars + len(content) > CONFIG["MAX_CONTEXT_CHARS"]:
                break
            source = doc.metadata.get('source', 'Doc')
            if '\\' in source:
                source = source.split('\\')[-1][:25]
            context_parts.append(f"[Documento {i+1}: {source}]\n{content}")
            total_chars += len(content)
        
        context = "\n\n".join(context_parts)
        system_prompt = self.SYSTEM_PROMPTS.get(intent, self.SYSTEM_PROMPTS["busqueda"])
        user_prompt = f"CONTEXTO:\n{context}\n\nPREGUNTA: {query}\n\nRESPUESTA:"
        
        # Intentar con Gemini primero (mejor para respuestas largas)
        response, success = self.gemini.generate(user_prompt, system_prompt, temperature=0.3, max_tokens=1000)
        
        if success:
            console.print(f"[green]✓ Respuesta generada [Gemini][/green]")
            self.trace_logger.log_event(
                agent="HybridRAGGenerator",
                action="generate",
                details={"intent": intent, "model": "Gemini", "context_chars": total_chars}
            )
            return response, "Gemini"
        
        # Fallback a Groq
        console.print(f"[yellow]⚠ Gemini falló, usando Groq fallback...[/yellow]")
        response, model = self.groq.invoke(system_prompt, user_prompt, max_tokens=800)
        
        if "ERROR" not in model:
            console.print(f"[green]✓ Respuesta generada [{model}][/green]")
        
        self.trace_logger.log_event(
            agent="HybridRAGGenerator",
            action="generate",
            details={"intent": intent, "model": model, "context_chars": total_chars}
        )
        
        return response, model


# =============================================================================
# VERIFICADOR CON LLM (Groq + Gemini fallback)
# =============================================================================
class HybridResponseVerifier:
    """
    Verificador de respuestas usando LLM real.
    Primario: Groq Llama 3.3 70B (pocos tokens de salida)
    Fallback: Gemini 2.0 Flash
    """
    
    SYSTEM_PROMPT = """Eres un evaluador crítico de respuestas. Verifica:
1. ¿La respuesta está respaldada por el contexto?
2. ¿Es coherente y lógica?
3. ¿Hay signos de alucinación (información no presente en el contexto)?
4. ¿Responde adecuadamente a la pregunta?

Responde SOLO con JSON:
{"es_valida": true/false, "confianza": 0.0-1.0, "problemas_detectados": [], "razonamiento": "..."}"""
    
    def __init__(self, trace_logger, groq_client: GroqWithFallback):
        self.trace_logger = trace_logger
        self.groq = groq_client
    
    def verify(self, query: str, response: str, docs: List) -> Dict[str, Any]:
        """Verificar respuesta usando LLM."""
        console.print(f"[yellow]✅ Verificando respuesta (LLM)...[/yellow]")
        
        # Si la respuesta es un error, no es válida
        if response.startswith("Error"):
            return {"es_valida": False, "confianza": 0, "razonamiento": "Error en generación", "modelo_usado": "N/A"}
        
        # Construir contexto resumido para verificación
        context_summary = "\n".join([doc.page_content[:300] for doc in docs[:3]])
        
        user_prompt = f"""Contexto (resumido):
{context_summary}

Pregunta: {query}

Respuesta a evaluar: {response[:500]}

Evalúa la respuesta."""
        
        llm_response, model_used = self.groq.invoke(self.SYSTEM_PROMPT, user_prompt, max_tokens=150)
        
        try:
            # Limpiar y parsear
            clean = llm_response.strip()
            if clean.startswith("```json"):
                clean = clean[7:]
            if clean.startswith("```"):
                clean = clean[3:]
            if clean.endswith("```"):
                clean = clean[:-3]
            
            result = json.loads(clean.strip())
            result["modelo_usado"] = model_used
            
            status = "[green]✓ Válida[/green]" if result.get("es_valida") else "[yellow]⚠ Revisar[/yellow]"
            console.print(f"{status} (conf: {result.get('confianza', 0):.2f}) [{model_used}]")
            
            self.trace_logger.log_event(
                agent="HybridResponseVerifier",
                action="verify",
                details=result
            )
            
            return result
            
        except json.JSONDecodeError:
            console.print(f"[yellow]⚠ Error parsing verificación, asumiendo válida[/yellow]")
            result = {"es_valida": True, "confianza": 0.7, "razonamiento": "Error parsing, asumida válida", "modelo_usado": model_used}
            self.trace_logger.log_event(agent="HybridResponseVerifier", action="verify_fallback", details=result)
            return result


# =============================================================================
# ORQUESTADOR HÍBRIDO
# =============================================================================
class HybridOrchestrator:
    """Orquestador del pipeline híbrido Groq + Gemini."""
    
    def __init__(self, classifier, retriever, generator, verifier, trace_logger, gemini_client, groq_client):
        self.classifier = classifier
        self.retriever = retriever
        self.generator = generator
        self.verifier = verifier
        self.trace_logger = trace_logger
        self.gemini = gemini_client
        self.groq = groq_client
    
    def process(self, query: str) -> Dict[str, Any]:
        """Procesar consulta completa."""
        self.trace_logger.start_trace(query)
        start_time = datetime.now()
        
        # Paso 1: Clasificar intención (Groq -> Gemini fallback)
        classification = self.classifier.classify(query)
        intent = classification["intencion"]
        
        # Paso 2: Rutear según intención
        if intent == "general":
            result = self._handle_general(query)
        else:
            result = self._handle_rag(query, intent)
        
        result["classification"] = classification
        result["processing_time"] = (datetime.now() - start_time).total_seconds()
        
        return result
    
    def _handle_general(self, query: str) -> Dict[str, Any]:
        """Manejar consulta general."""
        console.print("[cyan]💬 Procesando consulta general...[/cyan]")
        
        system = """Eres un experto en agricultura y ciencias agrícolas.
Responde de forma educativa, clara y concisa (máximo 300 palabras)."""
        
        # Gemini primario para respuestas generales
        response, success = self.gemini.generate(query, system, temperature=0.7, max_tokens=600)
        
        if success:
            model_used = "Gemini"
            console.print(f"[green]✓ Respuesta generada [{model_used}][/green]")
        else:
            # Fallback a Groq
            response, model_used = self.groq.invoke(system, query, max_tokens=500)
            if "ERROR" not in model_used:
                console.print(f"[green]✓ Respuesta generada [{model_used}][/green]")
        
        is_valid = not response.startswith("Error")
        
        return {
            "query": query,
            "intent": "general",
            "response": response,
            "documents_used": [],
            "verification": {"es_valida": is_valid, "confianza": 1.0 if is_valid else 0},
            "model_used": model_used
        }
    
    def _handle_rag(self, query: str, intent: str) -> Dict[str, Any]:
        """Manejar consulta RAG."""
        console.print(f"[cyan]🔄 Pipeline RAG ({intent})...[/cyan]")
        
        # Recuperar documentos (LOCAL)
        docs = self.retriever.retrieve(query, intent)
        
        if not docs:
            return {
                "query": query,
                "intent": intent,
                "response": "No se encontraron documentos relevantes.",
                "documents_used": [],
                "verification": {"es_valida": False, "confianza": 0},
                "model_used": "N/A"
            }
        
        # Generar respuesta (Gemini -> Groq fallback)
        response, gen_model = self.generator.generate(query, docs, intent)
        
        # Verificar (Groq -> Gemini fallback)
        verification = self.verifier.verify(query, response, docs)
        
        return {
            "query": query,
            "intent": intent,
            "response": response,
            "documents_used": [doc.metadata.get('source', f'doc_{i}')[-30:] for i, doc in enumerate(docs)],
            "verification": verification,
            "model_used": gen_model
        }


# =============================================================================
# EJECUTOR DE PRUEBAS
# =============================================================================
class RobustTestRunner:
    """Ejecutor de suite de pruebas."""
    
    def __init__(self, orchestrator):
        self.orchestrator = orchestrator
        self.results = []
    
    def run_test(self, test_id: int, query: str, expected_intent: str, description: str) -> Dict:
        """Ejecutar un caso de prueba."""
        console.print(f"\n{'═'*70}")
        console.print(f"[bold cyan]TEST #{test_id}[/bold cyan] │ {description}")
        console.print(f"[dim]Query: {query}[/dim]")
        console.print(f"[dim]Intención esperada: {expected_intent}[/dim]")
        console.print(f"{'═'*70}")
        
        start = datetime.now()
        
        try:
            result = self.orchestrator.process(query)
            duration = (datetime.now() - start).total_seconds()
            
            intent_match = result["intent"] == expected_intent
            is_valid = result.get("verification", {}).get("es_valida", False)
            response_ok = not result.get("response", "").startswith("Error")
            
            test_result = {
                "test_id": test_id,
                "description": description,
                "query": query,
                "expected_intent": expected_intent,
                "actual_intent": result["intent"],
                "intent_match": intent_match,
                "response_valid": is_valid and response_ok,
                "success": intent_match and response_ok,
                "duration_seconds": duration,
                "response_preview": result.get("response", "")[:300],
                "documents_count": len(result.get("documents_used", [])),
                "verification_confidence": result.get("verification", {}).get("confianza", 0),
                "model_used": result.get("model_used", "N/A"),
                "classifier_model": result.get("classification", {}).get("modelo_usado", "N/A"),
                "timestamp": start.isoformat()
            }
            
            # Mostrar resultado
            status_intent = "[green]✓[/green]" if intent_match else "[red]✗[/red]"
            status_valid = "[green]✓[/green]" if (is_valid and response_ok) else "[yellow]~[/yellow]"
            
            console.print(f"\n{status_intent} Intención: [magenta]{result['intent']}[/magenta] (esperada: {expected_intent})")
            console.print(f"{status_valid} Válida: {is_valid and response_ok} | Docs: {len(result.get('documents_used', []))} | Tiempo: {duration:.2f}s")
            console.print(f"[dim]Modelos: Clasificador={test_result['classifier_model']}, Generador={test_result['model_used']}[/dim]")
            
            if result.get("response") and not result["response"].startswith("Error"):
                border_style = "green" if (is_valid and response_ok) else "yellow"
                preview = result["response"][:400] + ("..." if len(result["response"]) > 400 else "")
                console.print(Panel(preview, title="📝 Respuesta", border_style=border_style))
            elif result.get("response"):
                console.print(Panel(result["response"][:150], title="⚠️ Error", border_style="red"))
            
        except Exception as e:
            duration = (datetime.now() - start).total_seconds()
            console.print(f"[red]✗ Error: {e}[/red]")
            test_result = {
                "test_id": test_id,
                "description": description,
                "query": query,
                "expected_intent": expected_intent,
                "actual_intent": "ERROR",
                "intent_match": False,
                "response_valid": False,
                "success": False,
                "duration_seconds": duration,
                "error": str(e),
                "timestamp": start.isoformat()
            }
        
        self.results.append(test_result)
        return test_result
    
    def generate_report(self, output_file: str = "test_results.json"):
        """Generar reporte completo."""
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, indent=2, ensure_ascii=False)
        
        console.print(f"\n[green]✓ Resultados guardados: {output_file}[/green]")
        
        # Tabla
        table = Table(title="📊 Resultados - Sistema Híbrido (Groq + Gemini)")
        table.add_column("ID", style="cyan", width=4)
        table.add_column("Descripción", style="white", width=18)
        table.add_column("Esperada", style="yellow", width=11)
        table.add_column("Detectada", style="magenta", width=11)
        table.add_column("Válida", style="green", width=6)
        table.add_column("Modelo Gen", style="blue", width=10)
        table.add_column("Estado", width=6)
        
        for r in self.results:
            status = "✅" if r["success"] else "❌"
            valid = "✓" if r.get("response_valid") else "~"
            model = r.get("model_used", "N/A")[:10]
            table.add_row(
                str(r["test_id"]),
                r["description"][:16],
                r["expected_intent"],
                r["actual_intent"],
                valid,
                model,
                status
            )
        
        console.print(table)
        
        # Estadísticas
        total = len(self.results)
        if total == 0:
            console.print("[yellow]No hay resultados[/yellow]")
            return
            
        intent_correct = sum(1 for r in self.results if r["intent_match"])
        responses_valid = sum(1 for r in self.results if r.get("response_valid"))
        full_success = sum(1 for r in self.results if r.get("success"))
        errors = sum(1 for r in self.results if "Error" in str(r.get("response_preview", "")))
        avg_time = sum(r["duration_seconds"] for r in self.results) / total
        
        # Contar modelos usados
        gemini_count = sum(1 for r in self.results if "Gemini" in str(r.get("model_used", "")))
        groq_count = sum(1 for r in self.results if "Groq" in str(r.get("model_used", "")))
        
        console.print(Panel(
            f"[bold]Total:[/bold] {total} pruebas\n"
            f"[bold green]Éxito completo:[/bold green] {full_success}/{total} ({full_success/total*100:.1f}%)\n"
            f"[bold]Clasificación correcta:[/bold] {intent_correct}/{total} ({intent_correct/total*100:.1f}%)\n"
            f"[bold]Respuestas válidas:[/bold] {responses_valid}/{total} ({responses_valid/total*100:.1f}%)\n"
            f"[bold]Con errores API:[/bold] {errors}\n"
            f"[bold]Tiempo promedio:[/bold] {avg_time:.1f}s\n"
            f"[bold cyan]Generación con Gemini:[/bold cyan] {gemini_count}\n"
            f"[bold cyan]Generación con Groq:[/bold cyan] {groq_count}",
            title="📈 Estadísticas - Sistema Híbrido",
            border_style="cyan"
        ))


# =============================================================================
# SUITE DE PRUEBAS PRINCIPAL
# =============================================================================
def run_hybrid_test_suite():
    """Ejecutar suite con arquitectura híbrida Groq + Gemini."""
    
    console.print(Panel(
        "[bold cyan]SUITE DE PRUEBAS - SISTEMA HÍBRIDO[/bold cyan]\n\n"
        "┌─────────────────────────────────────────────────────────┐\n"
        "│ [yellow]Clasificador:[/yellow]  Groq Llama 3.3 70B → Gemini fallback │\n"
        "│ [yellow]Generador:[/yellow]     Gemini 2.0 Flash → Groq fallback     │\n"
        "│ [yellow]Verificador:[/yellow]   Groq Llama 3.3 70B → Gemini fallback │\n"
        "│ [yellow]Embeddings:[/yellow]    SentenceTransformer (local)          │\n"
        "└─────────────────────────────────────────────────────────┘\n\n"
        "[green]✓ Todos los agentes usan LLMs reales (cumple requisitos)[/green]\n"
        "[green]✓ Fallback automático si un servicio falla[/green]",
        title="🚀 Inicializando",
        border_style="green"
    ))
    
    # Verificar API keys
    groq_key = os.getenv("GROQ_API_KEY")
    google_key = os.getenv("GOOGLE_API_KEY")
    
    if groq_key:
        console.print(f"[green]✓ GROQ_API_KEY configurada[/green]")
    else:
        console.print(f"[yellow]⚠ GROQ_API_KEY no configurada (solo Gemini)[/yellow]")
    
    if google_key:
        console.print(f"[green]✓ GOOGLE_API_KEY configurada[/green]")
    else:
        console.print(f"[red]✗ GOOGLE_API_KEY no configurada[/red]")
        if not groq_key:
            console.print("[red]ERROR: Se necesita al menos una API key[/red]")
            return
    
    # Inicializar clientes
    console.print("\n[bold]Inicializando clientes...[/bold]")
    
    gemini = GeminiClient(google_key) if google_key else None
    groq = GroqWithFallback(gemini, CONFIG["GROQ_MODEL_LARGE"], temperature=0.1) if gemini else None
    
    if not groq and not gemini:
        console.print("[red]ERROR: No hay clientes disponibles[/red]")
        return
    
    console.print("[green]✓ Clientes inicializados[/green]")
    
    # Embeddings locales
    embeddings = LocalEmbeddingsEngine()
    
    # Cargar índice FAISS
    trace_logger = TraceabilityLogger()
    indexer = DocumentIndexer(embeddings, trace_logger)
    vector_store = indexer.load_index("faiss_index")
    
    if vector_store is None:
        console.print("[yellow]⚠ Índice no encontrado, creando nuevo...[/yellow]")
        docs_path = os.getenv("DOCUMENTS_PATH", "./data")
        if os.path.exists(docs_path):
            documents = indexer.load_documents(docs_path)
            if documents:
                vector_store = indexer.process_and_index(documents)
                indexer.save_index("faiss_index")
            else:
                console.print("[red]ERROR: No hay documentos[/red]")
                return
        else:
            console.print(f"[red]ERROR: Directorio no existe: {docs_path}[/red]")
            return
    
    # Inicializar agentes híbridos
    classifier = HybridIntentClassifier(trace_logger, groq)
    retriever = SemanticDocumentRetriever(vector_store, trace_logger)
    generator = HybridRAGGenerator(trace_logger, gemini, groq)
    verifier = HybridResponseVerifier(trace_logger, groq)
    
    # Orquestador
    orchestrator = HybridOrchestrator(classifier, retriever, generator, verifier, trace_logger, gemini, groq)
    
    # Test runner
    runner = RobustTestRunner(orchestrator)
    
    # =========================================================================
    # CASOS DE PRUEBA
    # =========================================================================
    test_cases = [
        # Consultas generales (3)
        ("¿Qué es la agricultura sostenible?", "general", "Concepto general"),
        ("Explícame qué es la rotación de cultivos", "general", "Explicación teórica"),
        ("¿Qué es la agroecología?", "general", "Definición"),
        
        # Búsquedas en documentos (3)
        ("¿Qué técnicas de riego se mencionan en los documentos?", "busqueda", "Búsqueda técnicas"),
        ("¿Cuáles son los principales cultivos descritos?", "busqueda", "Búsqueda fáctica"),
        ("¿Qué desafíos ambientales se mencionan?", "busqueda", "Búsqueda problemas"),
        
        # Resúmenes (2)
        ("Resume el contenido principal sobre control de plagas", "resumen", "Resumen plagas"),
        ("Resume las estrategias de conservación del agua", "resumen", "Resumen agua"),
        
        # Comparaciones (2)
        ("Compara los diferentes métodos de fertilización", "comparacion", "Comparar métodos"),
        ("Contrasta la agricultura orgánica y la convencional", "comparacion", "Contraste sistemas"),
    ]
    
    delay = CONFIG["DELAY_BETWEEN_TESTS"]
    console.print(f"\n[bold]Ejecutando {len(test_cases)} pruebas...[/bold]")
    console.print(f"[dim]Delay entre pruebas: {delay}s | Fallback automático habilitado[/dim]\n")
    
    for idx, (query, expected, desc) in enumerate(test_cases, 1):
        runner.run_test(idx, query, expected, desc)
        if idx < len(test_cases):
            console.print(f"[dim]⏳ Esperando {delay}s...[/dim]")
            time.sleep(delay)
    
    # Generar reporte
    runner.generate_report()
    
    console.print("\n[bold green]✓ Suite completada[/bold green]")


# =============================================================================
# EJECUTAR
# =============================================================================
run_hybrid_test_suite()

╭─────────────────────────────────────────────── 🚀 Inicializando ────────────────────────────────────────────────╮
│ SUITE DE PRUEBAS - SISTEMA HÍBRIDO                                                                              │
│                                                                                                                 │
│ ┌─────────────────────────────────────────────────────────┐                                                     │
│ │ Clasificador:  Groq Llama 3.3 70B → Gemini fallback │                                                         │
│ │ Generador:     Gemini 2.0 Flash → Groq fallback     │                                                         │
│ │ Verificador:   Groq Llama 3.3 70B → Gemini fallback │                                                         │
│ │ Embeddings:    SentenceTransformer (local)          │                                                         │
│ └─────────────────────────────────────────────────────────┘                                                     │
│                                                                                                                 │
│ ✓ Todos los agentes usan LLMs reales (cumple requisitos)                                                        │
│ ✓ Fallback automático si un servicio falla                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ GROQ_API_KEY configurada

✓ GOOGLE_API_KEY configurada

Inicializando clientes...

✓ Clientes inicializados

...

🎯 Clasificando intención (LLM)...

⚠ Error Groq: 'Input to ChatPromptTemplate is missing variables , usando Gemini fallback...

⏳ Gemini rate limit, esperando 10s...

⏳ Gemini rate limit, esperando 20s...

⏳ Gemini rate limit, esperando 40s...